# Capstone Project 1 - Connecting to Data

In [1]:
import mysql.connector as mysql #if it gives error, use: pip install mysql.connector
import csv
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Creating the connection string to connect to INSAID's mysql db

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

In [3]:
print(dbconnectionstring)

In [4]:
# List all tables in the database

cursor.execute("SHOW TABLES")
tables=cursor.fetchall()
for table_name in tables:
    print(table_name)

('events_data',)
('gender_age_train',)
('phone_brand_device_model',)


In [5]:
# Table 1 - events_data - Get Column names

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "show columns from events_data"
cursor.execute(query)
result=cursor.fetchall()
dbconnectionstring.close()
result

[('event_id', 'bigint(20)', 'NO', '', None, ''),
 ('device_id', 'bigint(255)', 'YES', '', None, ''),
 ('timestamp', 'datetime', 'NO', '', None, ''),
 ('longitude', 'float(8,6)', 'YES', '', None, ''),
 ('latitude', 'float(8,6)', 'YES', '', None, ''),
 ('city', 'varchar(50)', 'NO', '', None, ''),
 ('state', 'varchar(50)', 'NO', '', None, '')]

In [6]:
# Table 2 - gender_age_train - Get Column names

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "show columns from gender_age_train"
cursor.execute(query)
result=cursor.fetchall()
dbconnectionstring.close()
result

[('device_id', 'bigint(255)', 'NO', '', None, ''),
 ('gender', 'varchar(2555)', 'NO', '', None, ''),
 ('age', 'int(255)', 'NO', '', None, ''),
 ('group', 'varchar(2555)', 'NO', '', None, '')]

In [7]:
# Table 3 - phone_brand_device_model - Get Column names

dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "show columns from phone_brand_device_model"
cursor.execute(query)
result=cursor.fetchall()
dbconnectionstring.close()
result

[('device_id', 'bigint(255)', 'NO', '', None, ''),
 ('phone_brand', 'varchar(2555)', 'NO', '', None, ''),
 ('device_model', 'varchar(2555)', 'NO', '', None, '')]

In [8]:
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "select count(*) from phone_brand_device_model"
cursor.execute(query)
pcount = cursor.fetchall()
dbconnectionstring.close()
pcount


[(87726,)]

In [9]:
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = "select count(*) from gender_age_train"
cursor.execute(query)
pcount = cursor.fetchall()
dbconnectionstring.close()
pcount


[(74645,)]

Query

In [17]:
# Get all records for the concerned States from events_data

if dbconnectionstring.is_connected():
    dbconnectionstring.close()
    
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

query = '''select *
             from events_data as ed
            where state in ('MadhyaPradesh', 'Goa', 'Chhattisgarh','Uttaranchal', 'JammuandKashmir', 'Nagaland')'''
cursor.execute(query)
events_data=cursor.fetchall()
ds_events_data= pd.DataFrame(events_data)
ds_events_data.columns=[i[0] for i in cursor.description]
dbconnectionstring.close()

In [18]:
ds_events_data.head()

,event_id,device_id,timestamp,longitude,latitude,city,state
0,10117,-5.323073e+18,2016-05-01 11:11:24,75.939545,22.769171,Indore,MadhyaPradesh
1,112369,-5.323073e+18,2016-05-04 22:47:19,75.939545,22.769171,Indore,MadhyaPradesh
2,139098,-5.323073e+18,2016-05-01 09:55:58,75.939545,22.769171,Indore,MadhyaPradesh
3,190983,-5.323073e+18,2016-05-04 22:51:03,75.939545,22.769171,Indore,MadhyaPradesh
4,249507,-5.323073e+18,2016-05-01 11:24:12,75.939545,22.769171,Indore,MadhyaPradesh


In [19]:
# Create a unique list of device ids from event data, to download only those records from the other 2 tables.

device_ids = ds_events_data['device_id'].unique()    # Extracts unique device ids
dids=device_ids.tolist()                           # Converts nparray to list
list_of_ids = [x for x in dids if str(x) != 'nan'] # Drop all 'nan' ids
len(list_of_ids)                                   # list the number of ids.

4297

In [22]:
# Download the records from phone_brand_device_model

if dbconnectionstring.is_connected():
    dbconnectionstring.close()
    
dbconnectionstring = mysql.connect(                     # Connection string used again, since it keeps disconnecting, so 
                      host = 'cpanel.insaid.co',        # executing this again, before every select statements.
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

format_strings = ','.join(['%s'] * len(list_of_ids))   # This needs to be added to pass the ids. if we have 5 ids, then it 
                                                       # will create 5, %s, positional parameters.

query = '''select * 
             from phone_brand_device_model
            where device_id in ( %s )''' % format_strings

cursor.execute(query, tuple(list_of_ids))
phone_data=cursor.fetchall()
ds_phone_data = pd.DataFrame(phone_data)
ds_phone_data.columns=[i[0] for i in cursor.description]
dbconnectionstring.close()

In [23]:
ds_phone_data.head()

,device_id,phone_brand,device_model
0,2313145512701915151,vivo,Xplay
1,-1539979963116391306,小米,GN708W
2,-7731743086143077949,小米,红米
3,-4370696809239943432,小米,MI 2S
4,2959524852145775517,三星,A850


In [24]:
# Download the records from gender_age_train

if dbconnectionstring.is_connected():
    dbconnectionstring.close()
    
dbconnectionstring = mysql.connect(
                      host = 'cpanel.insaid.co',
                      user = 'student',
                      passwd = 'student',
                      database = 'Capstone1' )

cursor = dbconnectionstring.cursor()

format_strings = ','.join(['%s'] * len(list_of_ids))

#list_ids = tuple(','.join(['%s'] * len(ldevice_ids)))

query = '''select * 
             from gender_age_train
            where device_id in ( %s )''' % format_strings

cursor.execute(query,tuple(list_of_ids))
user_data=cursor.fetchall()
ds_user_data = pd.DataFrame(user_data)
ds_user_data.columns=[i[0] for i in cursor.description]
dbconnectionstring.close()

In [25]:
ds_user_data.head()

,device_id,gender,age,group
0,2313145512701915151,M,34,M32-38
1,-1539979963116391306,M,31,M29-31
2,-7731743086143077949,M,35,M32-38
3,-4370696809239943432,M,36,M32-38
4,2959524852145775517,M,30,M29-31


In [26]:
# Write all records from the data, to excel files

ds_events_data.to_excel('events_data.xlsx')
ds_phone_data.to_excel('phone_data.xlsx')
ds_user_data.to_excel('user_data.xlsx')

In [27]:
phone_brand_list = ds_phone_data['phone_brand'].unique()

In [28]:
phone_brand_list.shape

(60,)

In [ ]:
for i in range(len(phone_brand_list)):
   # translator = Translator()
   # translated= translator.translate(phone_brand_list[i], dest='en')
    print (phone_brand_list[i]) #, ' - ', translated)

In [30]:
phone_device_model = ds_phone_data['device_model'].unique()
phone_device_model = pd.DataFrame(phone_device_model)
phone_device_model.to_excel('phone_device_model_data.xlsx')

In [46]:
df_model_typ = pd.read_excel("phone_model_trans.xlsx")
df_model_typ.head(5)

,model,model_eng
0,Xplay,Xplay
1,GN708W,GN708W
2,红米,Red rice
3,MI 2S,MI 2S
4,A850,A850


In [54]:
dp = ds_phone_data.copy()

In [48]:
df_tpb = pd.read_excel("phone_brand_translated.xlsx")
df_tpb.head(5)

,phone_brand,phone_brand_eng
0,酷派,Cool
1,魅族,Meizu
2,努比亚,Nubia
3,三星,Samsung
4,纽曼,Newman


In [49]:
def trans_brand(row):
    for i in df_tpb.index:
        if df_tpb['phone_brand'][i] == row['phone_brand'] : 
            return df_tpb['phone_brand_eng'][i]
        
    return "Other"

In [55]:
dp['phone_brand_eng'] = dp.apply(lambda row: trans_brand(row), axis=1 )
dp.head(5)

,device_id,phone_brand,device_model,phone_brand_eng
0,2313145512701915151,vivo,Xplay,vivo
1,-1539979963116391306,小米,GN708W,Millet
2,-7731743086143077949,小米,红米,Millet
3,-4370696809239943432,小米,MI 2S,Millet
4,2959524852145775517,三星,A850,Samsung


In [64]:
def trans_model(row):
    try:
        for i in df_model_typ.index:
            if df_model_typ['model'][i] == row['device_model'] :
                return df_model_typ['model_eng'][i]
    except Exception as e:
        print(str(e))
        return "df_model_typ['model'][i]"
        
    return "df_model_typ['model'][i]"

In [63]:
dp['device_model_eng'] = dp.apply(lambda row: trans_model(row), axis=1 )
dp.head(5)

,device_id,phone_brand,device_model,phone_brand_eng,device_model_eng
0,2313145512701915151,vivo,Xplay,vivo,Xplay
1,-1539979963116391306,小米,GN708W,Millet,GN708W
2,-7731743086143077949,小米,红米,Millet,Red rice
3,-4370696809239943432,小米,MI 2S,Millet,MI 2S
4,2959524852145775517,三星,A850,Samsung,A850


joining datasets

In [71]:
din = ds_events_data.set_index('device_id').join(dp.set_index('device_id'))
din.head()

ValueError: values should be unique if labels is not None

In [67]:
din.shape

(261097, 12)

In [69]:
ds_events_data.shape

(261097, 7)

In [73]:
ds_events_data.describe()

,event_id,device_id,longitude,latitude
count,2.610970e+05,2.610460e+05,261034.000000,261034.000000
mean,1.625966e+06,-4.377374e+16,76.704918,23.258085
std,9.401508e+05,5.153156e+18,2.333525,2.219976
min,1.011700e+04,-9.222755e+18,12.567400,15.291265
25%,8.136240e+05,-4.374356e+18,75.897141,22.750061
50%,1.624810e+06,6.577099e+16,75.925995,22.782286
75%,2.438915e+06,4.095288e+18,75.956451,22.811922
max,3.252939e+06,9.220026e+18,94.361549,41.871899


In [74]:
ds_events_data.isnull().sum()

event_id      0
device_id    51
timestamp     0
longitude    63
latitude     63
city          0
state         0
dtype: int64

In [76]:
ds_events_data[ds_events_data['device_id'].isnull()]

,event_id,device_id,timestamp,longitude,latitude,city,state
48308,84091,NaN,2016-05-02 13:28:09,75.882957,22.814520,Indore,MadhyaPradesh
48311,93602,NaN,2016-05-03 11:57:22,75.882957,22.814520,Indore,MadhyaPradesh
48379,356056,NaN,2016-05-02 13:48:16,75.882957,22.814520,Indore,MadhyaPradesh
48427,558444,NaN,2016-05-02 18:58:23,75.882957,22.814520,Indore,MadhyaPradesh
48490,850678,NaN,2016-05-07 15:40:10,75.882957,22.814520,Indore,MadhyaPradesh
48520,1037766,NaN,2016-05-03 12:00:46,75.882957,22.814520,Indore,MadhyaPradesh
48567,1228197,NaN,2016-05-02 13:32:29,75.882957,22.814520,Indore,MadhyaPradesh
48590,1350986,NaN,2016-05-01 22:11:23,75.882957,22.814520,Indore,MadhyaPradesh
48608,1376079,NaN,2016-05-03 16:14:56,75.882957,22.814520,Indore,MadhyaPradesh
48660,1603987,NaN,2016-05-04 10:57:10,75.882957,22.814520,Indore,MadhyaPradesh


In [80]:
ds_events_data[ds_events_data['longitude'] == 75.923332]

,event_id,device_id,timestamp,longitude,latitude,city,state
114945,12819,7.597736e+18,2016-05-01 13:06:16,75.923332,22.777781,Indore,MadhyaPradesh
114946,14548,7.597736e+18,2016-05-02 14:49:42,75.923332,22.777781,Indore,MadhyaPradesh
114947,14600,7.597736e+18,2016-05-02 14:41:25,75.923332,22.777781,Indore,MadhyaPradesh
114948,14601,7.597736e+18,2016-05-02 14:44:06,75.923332,22.777781,Indore,MadhyaPradesh
114949,17218,7.597736e+18,2016-05-01 16:24:02,75.923332,22.777781,Indore,MadhyaPradesh
114950,20521,7.597736e+18,2016-05-04 18:51:10,75.923332,22.777781,Indore,MadhyaPradesh
114951,20958,7.597736e+18,2016-05-06 18:41:40,75.923332,22.777781,Indore,MadhyaPradesh
114952,23830,7.597736e+18,2016-05-05 20:27:53,75.923332,22.777781,Indore,MadhyaPradesh
114953,23927,7.597736e+18,2016-05-05 19:59:17,75.923332,22.777781,Indore,MadhyaPradesh
114954,26580,7.597736e+18,2016-05-03 22:22:55,75.923332,22.777781,Indore,MadhyaPradesh


In [79]:
ds_events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261097 entries, 0 to 261096
Data columns (total 7 columns):
event_id     261097 non-null int64
device_id    261046 non-null float64
timestamp    261097 non-null datetime64[ns]
longitude    261034 non-null float64
latitude     261034 non-null float64
city         261097 non-null object
state        261097 non-null object
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 13.9+ MB


In [81]:
de = ds_events_data.copy()

In [84]:
ds_events_data['longitude'].unique().shape

(4192,)

In [85]:
ds_events_data['latitude'].unique().shape

(4280,)

In [86]:
ds_events_data['device_id'].unique().shape

(4298,)

In [87]:
ds=ds_events_data.copy()

In [88]:
ds['combined_geo'] = ds.apply(lambda row: str(row['longitude']) + str(row['latitude']), axis=1 )

In [90]:
ds['combined_geo'].unique().shape

(4301,)

In [108]:
#Map events data 

import folium
start_coord = [0,0]                                                           # starting co-ordinates
maps = folium.Map(location=start_coord, titles='OpenStreetMap', zoom_start=4)
df_map_series = pd.notnull(ds_events_data['longitude'])                       # create a series that has boolean values if nulls are present or not.
df_map = ds_events_data[df_map_series] #.iloc[0:5000, :]                      #un-comment .iloc, if sample data has to be mapped.
for i in range(0,len(df_map)):
    folium.CircleMarker(
                            location = [df_map.iloc[i]['latitude'], df_map.iloc[i]['longitude']],
                  popup="Location: {}, {}<br>Device Id: {}".
                  format(df_map.iloc[i]['latitude'], df_map.iloc[i]['longitude'],df_map.iloc[i]['device_id'] )
                            ).add_to(maps)
maps.save('events_map.html')                                                  # Map will be saved to this file, same folder as this notebook.